# Summer Olympics

## Scenario
Just recently the Summer Olympics concluded and all the newspapers were printing the medal rankings. There seem to be different methods on how to rank the table, used in different countries. Apparently there is some room for interpretation.

## Tasks
The sporting associations of the *** countries created a task force in order to investigate if the used medal ranking reflects their achievements. They supplied us with the data they have found on the countries and their main concerns are the following:

- How did the *** countries perform and who has room for improvement and why?
- Are the official rankings "fair" or do you think that e.g. the top 5 achievers should look different?

The questions are being asked as ultimately they are looking who is the candidate within the group to go and learn from.

In [1]:
import pandas as pd

df_countries = pd.read_csv('data/countries_data.tsv', sep='\t', header=0)
df_delegations = pd.read_csv('data/delegations_data.tsv', sep='\t', header=0)
df_medals = pd.read_csv('data/tokio2021_medal_counts.tsv', sep='\t', header=0)

# Data cleansing
df_medals['country'] = df_medals['country'].str.replace('The ', '')

In [2]:
# Join datasets
df = df_medals.merge(df_delegations, on='country', how='left')
df = df.merge(df_countries, on='country', how='left')

In [4]:
# Feature engineering

# Assumption 1: all medalists are winners
df['total_medals'] = df[['gold', 'silver', 'bronze']].sum(axis=1)

# Assumption 2: people in wealthier countries have more spare time, therefore more time for sports activities
df['GDP_K_per_capita'] = df['GDP_M_USD'] / df['population'] * 1000

# Assumption 3: top athletes are only once in a million, therefore larger countries benefit more
df['athletes_ratio'] = df.athletes / df.population

# reverse and normalize data
df['GDP_K_per_capita_norm'] = df['GDP_K_per_capita'] / df['GDP_K_per_capita'].max()
df['athletes_ratio_norm'] = df['athletes_ratio'] / df['athletes_ratio'].max()
df['GDP_K_per_capita_norm2'] = 1 / df['GDP_K_per_capita_norm']
df['GDP_K_per_capita_revnorm'] = df['GDP_K_per_capita_norm2'] / df['GDP_K_per_capita_norm2'].max()

df = df.drop(['GDP_K_per_capita_norm', 'GDP_K_per_capita_norm2'], axis = 1)

df['fair'] = df['total_medals'] * df['GDP_K_per_capita_revnorm'] * df['athletes_ratio_norm']

df.sort_values(['fair'], ascending=False).head(15)

,country,gold,silver,bronze,ico_code,athletes,GDP_M_USD,population,total_medals,GDP_K_per_capita,athletes_ratio,athletes_ratio_norm,GDP_K_per_capita_revnorm,fair
20,Jamaica,4,1,4,JAM,62.0,14600.0,2734093.0,9,5.339979,0.000023,0.152382,0.149281,0.204731
14,Hungary,6,7,7,HUN,173.0,176543.0,9730772.0,20,18.142754,0.000018,0.119469,0.043938,0.104985
12,New Zealand,7,6,7,NZL,228.0,243332.0,5131249.0,20,47.421593,0.000044,0.298585,0.016810,0.100385
43,Ukraine,1,6,12,UKR,156.0,164593.0,41362393.0,19,3.979291,0.000004,0.025344,0.200327,0.096465
32,Georgia,2,5,1,GEO,35.0,16163.0,3728573.0,8,4.334902,0.000009,0.063079,0.183893,0.092798
4,Russia,20,28,23,ROC,343.0,1710734.0,146171015.0,71,11.703647,0.000002,0.015768,0.068112,0.076256
59,Fiji,1,0,1,FIJ,32.0,4586.0,898402.0,2,5.104619,0.000036,0.239351,0.156164,0.074756
5,Australia,17,7,22,AUS,490.0,1617543.0,25875521.0,46,62.512480,0.000019,0.127252,0.012752,0.074645
27,Serbia,3,1,5,SRB,87.0,60435.0,6871547.0,9,8.794963,0.000013,0.085079,0.090638,0.069402
70,Mongolia,0,1,3,MGL,43.0,14233.0,3400736.0,4,4.185270,0.000013,0.084967,0.190468,0.064734


### How did the *** countries perform?
Let's have a look at Switzerland, Germany, Spain, Romania, Slovakia, and the Philippines

In [9]:
selection = ['Switzerland', 'Germany', 'Spain', 'Romania', 'Slovakia', 'Philippines']
df[df.country.isin(selection)].sort_values(['fair'], ascending=False).head(10)

,country,gold,silver,bronze,ico_code,athletes,GDP_M_USD,population,total_medals,GDP_K_per_capita,athletes_ratio,athletes_ratio_norm,GDP_K_per_capita_revnorm,fair
21,Spain,3,8,6,ESP,337.0,1461552.0,47394223.0,17,30.838189,7.110571e-06,0.047782,0.025850,0.020997
8,Germany,10,11,16,GER,422.0,4319286.0,83129285.0,37,51.958657,5.076430e-06,0.034113,0.015342,0.019364
23,Switzerland,3,4,6,SUI,117.0,824734.0,8696088.0,13,94.839657,1.345433e-05,0.090410,0.008405,0.009879
45,Romania,1,3,0,ROU,104.0,289130.0,19186201.0,4,15.069685,5.420562e-06,0.036425,0.052898,0.007707
50,Slovakia,1,2,1,SVK,41.0,117664.0,5459781.0,4,21.551048,7.509459e-06,0.050462,0.036989,0.007466
49,Philippines,1,2,1,PHI,19.0,402638.0,110911008.0,4,3.630280,1.713085e-07,0.001151,0.219586,0.001011
